In [1]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
# Personnal import 
from utils.utilities_DL import match_period_coverage_with_netmob,get_small_ds
from trainer import Trainer
from constants.config import get_args,update_modif
from constants.paths import folder_path,file_name,SAVE_DIRECTORY
from K_fold_validation.K_fold_validation import KFoldSplitter
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from plotting.plotting_bokeh import plot_bokeh

# Hp Tuning
from HP_tuning.ray_search_space import get_search_space_ray 
from HP_tuning.ray_config import get_ray_config
import ray 
from ray import tune 

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

'pynvml' is not available on this environment.


In [3]:
# Load config
model_name = 'STGCN' #'CNN'
args = get_args(model_name)

# Modification : 
args.K_fold = 5
args.ray = True
args.W = 0  # IMPORTANT AVEC NETMOB
args.epochs = 1
args.loss_function_type = 'MSE' # 'quantile'

args = update_modif(args)

# Coverage Period : 
small_ds = False
coverage = match_period_coverage_with_netmob(file_name)
(coverage,args) = get_small_ds(small_ds,coverage,args)

# Choose DataSet and VisionModel if needed: 
dataset_names = ['netmob'] # ['calendar','netmob'] #['subway_in','netmob','calendar']
vision_model_name = 'ImageAvgPooling'  # 'ImageAvgPooling'  #'FeatureExtractor_ResNetInspired' #'MinimalFeatureExtractor',

# Train and Evaluate Model: 
mod_plot = 1 # bokeh plotting every epoch 

# Load K-fold subway-ds 
K_fold_splitter = KFoldSplitter(dataset_names,args,coverage,folder_path,file_name,vision_model_name)
K_subway_ds,args_embedding,args_vision,dic_class2rpz = K_fold_splitter.split_k_fold()

Model: STGCN, K_fold = 5
!!! Loss function: MSE 
Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-05-31 23:45:00

U size:  torch.Size([6719, 40, 7]) Utarget size:  torch.Size([6719, 40, 1])
U_train size:  torch.Size([3912, 40, 7]) Utarget_train size:  torch.Size([3912, 40, 1])
U_valid size:  torch.Size([1304, 40, 7]) Utarget_valid size:  torch.Size([1304, 40, 1])
U_test size:  torch.Size([1310, 40, 7]) Utarget_test size:  torch.Size([1310, 40, 1])
U_train min:  tensor(0.) U_train max:  tensor(10798.)
U_valid min:  tensor(0.) U_valid max:  tensor(1405.)
U_test min:  tensor(0.) U_test max:  tensor(1760.)
Load des données NetMob .pt impossible. Création d'un random Tensor
Init NetMob Dataset:  torch.Size([7392, 40, 2, 8, 8])
Number of Nan Value:  tensor(0)
Total Number of Elements:  37847040 


U size:  torch.Size([6719, 40, 2, 8, 8, 7]) Utarget size:  torch.Size([6719, 40, 2, 8, 8, 1])
U_train size:  torch.Size([3912, 40, 2, 8, 8, 7]) Utarget_train size:  torch.Size([3

In [4]:
subway_ds = K_subway_ds[0]
# Load Model:
model = load_model(args,args_embedding,dic_class2rpz,args_vision)

# Load Optimizer, Scheduler, Loss function: 
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)

trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,args_embedding  =args_embedding,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
# Train Model 
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

pi,pi_cqr = plot_bokeh(trainer,args)


PREDICTION WILL BE BASED SOLELY ON CONTEXTUAL DATA !

number of total parameters: 207233
number of trainable parameters: 207233

start training


2024-08-13 19:59:26,578	WARNING session.py:91 -- Session not detected. You should not be calling `report` outside `tuner.fit()` or while using the class API. 
2024-08-13 19:59:26,579	WARNING session.py:97 --   File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/romainrochas/opt/anaconda3/envs/tuning/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/romainrochas/opt/anaconda3/envs/tuning/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/romainrochas/opt/anaconda3/envs/tuning/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/Users/romainrochas/opt/anaconda3/envs/tuning/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/Users/romainrochas/opt/anaconda3/envs

epoch: 0 
 min\epoch : 0.06
>>> Training complete in: 0:00:03.383434
>>> Training performance time: min 0.06992793083190918 avg 0.07291078567504883 seconds (+/- 0.004574284464341534)
>>> Loading performance time: min 0.00341796875 avg 0.003700292471683387 seconds (+/- 0.00022829645098773458)
>>> Forward performance time: 0.020667278405391808 seconds (+/- 0.0011608639620860283)
>>> Backward performance time: 0.053893212116125855 seconds (+/- 0.004221733984341022)
>>> Plotting performance time: nan seconds (+/- nan)
>>> PI-tracking performance time: nan seconds (+/- nan)
>>> Scheduler-update performance time: nan seconds (+/- nan)
>>> Validation time: 0:00:00.368964
Proportion of time consumed for Loading: 4.3%
Proportion of time consumed for Forward: 24.3%
Proportion of time consumed for Backward: 62.0%
Proportion of time consumed for Plotting: 0.0%
Proportion of time consumed for CheckPoint Saving: 0.0%
Proportion of time consumed for Tracking PI: 9.4%
Proportion of time consumed for U

Loading BokehJS ...

In [ ]:
from utils.utilities_DL import get_MultiModel_loss_args_emb_opts,load_init_trainer

## Hyper Parameter Tuning sur le Fold 0
def load_trainer(config,folder_path,file_name,args):

    for key, value in config.items():
        if hasattr(args, key):
            setattr(args, key, value)

    Datasets,DataLoader_list,dic_class2rpz,nb_words_embedding,time_slots_labels,dic_rpz2class = load_init_trainer(folder_path,file_name,args)
    (loss_function,Model_list,Optimizer_list,Scheduler_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = len(Datasets[0].columns))
    dataset,dataloader,model,optimizer,scheduler = Datasets[0],DataLoader_list[0],Model_list[0],Optimizer_list[0],Scheduler_list[0]


    trainer = Trainer(dataset,model,dataloader,
                    args,optimizer,loss_function,scheduler = scheduler,
                    args_embedding=args_embedding,dic_class2rpz=dic_class2rpz)
    return(trainer)

def Train_with_tune(config,folder_path,file_name,args):
    trainer = load_trainer(config,folder_path,file_name,args)
    result_df = trainer.train_and_valid()

    
def run_tuning_and_save_results(args,num_samples):
    config = get_search_space_ray(args)
    ray_scheduler,ray_search_alg,resources_per_trial,num_gpus,max_concurrent_trials,num_cpus = get_ray_config(args)
    
    def trainer(config):
        return(Train_with_tune(config,folder_path,file_name,args))
        
    if ray.is_initialized:
        ray.shutdown()
        ray.init(num_gpus=num_gpus,num_cpus=num_cpus)

        
    analysis = tune.run(
            trainer,
            config=config,
            num_samples=num_samples,  # Increase num_samples for more random combinations
            resources_per_trial = resources_per_trial,
            max_concurrent_trials = max_concurrent_trials,
            scheduler = ray_scheduler,
            search_alg = ray_search_alg,
        )

    name_save = f"HyperparameterTuning/Htuning_ray_analysis_{args.model_name}_loss{args.loss_function_type}_TE_{args.time_embedding}"
    analysis.results_df.to_csv(f'{name_save}.csv')

In [ ]:
# ==== GET PARAMETERS ====
# Load config
model_name = 'STGCN'  #'CNN'
args = get_args(model_name)

# Classic Modification : 
args.epochs = 1
args.loss_function_type = 'MSE' # 'quantile'
# ...

# Modification pour HyperParameterTuning:
args.K_fold = 5
args.ray = True
# ...

# Update Modif 
args = update_modif(args,name_gpu='cuda:0')
# ...

run_tuning_and_save_results(args,num_samples=1000)